# Short circuits in MV line

In [19]:
import numpy as np
from pydgrid import grid
from pydgrid.plot_bokeh import plot_results

![](./png/mv_short_circuits.png)

## System

In [20]:
data = {
"transformers":[
        {"bus_j": "B0",  "bus_k": "B1",  "S_n_kVA": 32000.00, "U_j_kV":66.0, "U_k_kV":18.0,
        "R_cc_pu": 0.01, "X_cc_pu":0.08, "connection": "Dyn11",
        "conductors_1": 3, "conductors_2": 4}
        ],
"lines":[
        {"bus_j": "B1",  "bus_k": "B2",  "code": "mv_al_120", "m": 20000.0}
        ],
"buses":[
        {"bus": "B0",  "pos_x": 0,  "pos_y": 0, "units": "m", "U_kV":66.0},
        {"bus": "B1",  "pos_x":30,  "pos_y": 0, "units": "m", "U_kV":18.0},
        {"bus": "B2",  "pos_x":200, "pos_y": 0, "units": "m", "U_kV":18.0}
        ],
"grid_formers":[
        {"bus": "B0",
        "bus_nodes": [1, 2, 3], "deg": [0, -120, -240],
        "kV": [38.105, 38.105, 38.105]}
        ],
"grid_feeders":[{"bus": "B2","bus_nodes": [1, 2, 3],
                "kW": [0, 0, 0], "kvar": [0,0,0],
                "kA": [-0.082,-0.082,-0.082], "phi_deg":[0, 0, 0]}
                ],   
"line_codes":
        {"mv_al_35":  {"R1":1.093, "X1":0.148, "R0":1.093, "X0":0.148}, # no hay dato de X1, se toma del 50 mm2
         "mv_al_50":  {"R1":0.8,   "X1":0.148, "R0":0.8,   "X0":0.148},
         "mv_al_70":  {"R1":0.558, "X1":0.137, "R0":0.558, "X0":0.137},
         "mv_al_95":  {"R1":0.403, "X1":0.129, "R0":0.403, "X0":0.129},
         "mv_al_120": {"R1":0.321, "X1":0.123, "R0":0.321, "X0":0.123},
         "mv_al_150": {"R1":0.262, "X1":0.118, "R0":0.262, "X0":0.118},
         "mv_al_185": {"R1":0.209, "X1":0.113, "R0":0.209, "X0":0.113},
         "mv_al_300": {"R1":0.128, "X1":0.105, "R0":0.128, "X0":0.105}
         },
"shunts":[
        {"bus": "B1" , "R": 1e3, "X": 0.0, "bus_nodes": [4,0]},
        ]
}

In [21]:
grid_1 = grid()
grid_1.read(data)  # Load data
grid_1.pf()  # solve power flow
p=plot_results(grid_1)

## Three phase short circuit

![](./png/mv_short_circuits_3ph.png)  

In [11]:
grid_1 = grid()
data['grid_feeders'] = [{"bus": "B2","bus_nodes": [1, 2, 3],
                "kW": [0, 0, 0], "kvar": [0,0,0],
                "kA": [-0.0,-0.0,-0.0], "phi_deg":[0, 0, 0]}
                ]
data['shunts'] = [
        {"bus": "B1" , "R": 1000, "X": 0.0, "bus_nodes": [1,0]},
        {"bus": "B1" , "R": 1e-8, "X": 0.0, "bus_nodes": [1,2]}, # applied fault
        {"bus": "B1" , "R": 1e-8, "X": 0.0, "bus_nodes": [2,3]}, # applied fault
]
grid_1.read(data)  # Load data
grid_1.pf()  # solve power flow
p=plot_results(grid_1)

In [12]:
S_cc = 400e6
U_b = 18e3
Xpu = 0.08
I = S_cc/(np.sqrt(3)*U_b)
S_b = U_b*Xpu*np.sqrt(3)*I
print(f'I_cc = {I/1000:0.2f} kA')
print(f'S_b = {S_b/1000:0.2f} kVA')


I_cc = 12.83 kA
S_b = 32000.00 kVA


## Phase ground fault (isolated neutral)

![](./png/mv_short_circuits_ph_g_isolated_n.png)  

In [13]:
data['grid_feeders'] = [{"bus": "B2","bus_nodes": [1, 2, 3],
                "kW": [0, 0, 0], "kvar": [0,0,0],
                "kA": [-0.0,-0.0,-0.0], "phi_deg":[0, 0, 0]}
                ]

data['shunts'] = [
        {"bus": "B1" , "R": 1e8, "X": 0.0, "bus_nodes": [1,0]},
        {"bus": "B1" , "R": 1e-8, "X": 0.0, "bus_nodes": [1,0]}, # applied fault
]


In [14]:
grid_ph_g = grid()
grid_ph_g.read(data)  # Load data
grid_ph_g.pf()  # solve power flow
p=plot_results(grid_ph_g)

## Phase ground fault (grounded neutral)

![](./png/mv_short_circuits_ph_g_n.png)  

In [15]:
data['grid_feeders'] = [{"bus": "B2","bus_nodes": [1, 2, 3],
                "kW": [0, 0, 0], "kvar": [0,0,0],
                "kA": [-0.0,-0.0,-0.0], "phi_deg":[0, 0, 0]}
                ]
data['shunts'] = [
        {"bus": "B1" , "R": 1e-12, "X": 0.0, "bus_nodes": [4,0]},
        {"bus": "B1" , "R": 1e-12, "X": 0.0, "bus_nodes": [1,0]}, # applied fault
]

In [16]:
grid_ph_n = grid()
grid_ph_n.read(data)  # Load data
grid_ph_n.pf()  # solve power flow
p=plot_results(grid_ph_n)

## Phase ground fault (grounded neutral with impedance)

![](./png/mv_short_circuits_ph_g_z_n.png)  

In [17]:
data['grid_feeders'] = [{"bus": "B2","bus_nodes": [1, 2, 3],
                "kW": [0, 0, 0], "kvar": [0,0,0],
                "kA": [-0.0,-0.0,-0.0], "phi_deg":[0, 0, 0]}
                ]
data['shunts'] = [
        {"bus": "B1" , "R": 3.5, "X": 0.0, "bus_nodes": [4,0]},
        {"bus": "B1" , "R": 1e-12, "X": 0.0, "bus_nodes": [1,0]}, # applied fault
]

In [18]:
grid_ph_n = grid()
grid_ph_n.read(data)  # Load data
grid_ph_n.pf()  # solve power flow
p=plot_results(grid_ph_n)

Run this notebook in binder:

[![Documentation Status](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/pydgrid/pydgrid/master?filepath=/docs/jupyter_notebooks/mv_short_circuit.ipynb)